**FastAI book chapter 5, Pet Breeds**
Image classification, attempting to get it to run on my GTX 1080Ti

Need to install fastai.  Probably could add this to the conda environment if I find that I need to do this often

In [1]:
# installing/upgrading fastai2 
!pip install --upgrade fastai

In [2]:
%matplotlib inline
from fastai import *
from fastai.vision import *
from fastai.vision.all import *

Check to ensure that my GPU is visible and selectable

In [3]:
# Test to ensure that my cuda is still visible
import torch as torch
torch.cuda.is_available()

True

In [4]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080 Ti'

Now set up the fastbook itself